In [1]:
!pip install polars --no-index --find-links=file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
!pip install duckdb --no-index --find-links=file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg

Looking in links: file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
Processing /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg/polars-0.20.16-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Looking in links: file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg
Processing /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg/duckdb-0.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [2]:
import polars as pl
import duckdb as dd
import os
from datetime import datetime
from joblib import Parallel, delayed
from tqdm import tqdm
from pathlib import Path

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

import tensorflow as tf
# import tensorflow_io as tfio
from tensorflow import keras

E0000 00:00:1730536234.717805      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
D1102 08:30:34.726051889      13 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D1102 08:30:34.726071786      13 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D1102 08:30:34.726074956      13 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D1102 08:30:34.726077317      13 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D1102 08:30:34.726079665      13 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defa

In [3]:
# detect TPUs
tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

print("Number of accelerators: ", tpu_strategy.num_replicas_in_sync)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1730536308.162652      13 service.cc:145] XLA service 0x57bb60151b50 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730536308.162708      13 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1730536308.162713      13 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1730536308.162716      13 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1730536308.162718      13 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1730536308.162721      13 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1730536308.162724      13 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1730536308.162727      13 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1730536308.162729      13 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [4]:
from multiprocessing import cpu_count
n_cores = cpu_count()
print(f'Number of Logical CPU cores: {n_cores}')

Number of Logical CPU cores: 96


In [5]:
Test = True

if Test:
    num_jobs = 92
else:
    num_jobs = 4

config_dict = {'train_csv_path': '/kaggle/input/child-mind-institute-problematic-internet-use/train.csv'}
config_dict['test_csv_path'] = '/kaggle/input/child-mind-institute-problematic-internet-use/test.csv'
config_dict['train_parquet_root_path'] = '/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/'
config_dict['test_parquet_root_path'] = '/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet/'

In [6]:
train_data = pl.read_csv(config_dict['train_csv_path'], low_memory=True)
# train_data.describe()
train_data_id_list = pl.Series(train_data.select(pl.col('id'))).to_list()
# len(train_data_id_list)

# actigraphy_file_list = []
actigraphy_id_list = []

for dirname, _, filenames in os.walk(config_dict['train_parquet_root_path']):
    for filename in filenames:
        full_parquet_file_path = os.path.join(dirname, filename)
        # actigraphy_file_list.append(full_parquet_file_path)
        actigraphy_id_list.append(full_parquet_file_path.split('/')[-2].replace('id=',''))
        
final_id_list = list(set(actigraphy_id_list).intersection(train_data_id_list))

print(len(final_id_list))
print(len(train_data_id_list))

996
3960


In [7]:
categorical_colms = ['Basic_Demos-Enroll_Season','CGAS-Season','Physical-Season','Fitness_Endurance-Season','FGC-Season'\
                    ,'BIA-Season','PCIAT-Season','PreInt_EduHx-Season','PAQ_A-Season','PAQ_C-Season','SDS-Season']

for cat_col in categorical_colms:
    cat_vals_list = pl.Series(train_data.select(pl.col(cat_col))).to_list()
    encoded_vals_list = le.fit_transform(cat_vals_list)
    encoded_col = "encoded_{0}".format(cat_col)
    train_data = train_data.with_columns(pl.Series(name=encoded_col, values=encoded_vals_list))
    train_data = train_data.drop([cat_col])

In [12]:
# Calculate the approximate size of each smaller list
chunk_size = len(train_data_id_list) // num_jobs
remainder = len(train_data_id_list) % num_jobs

# Create the num_jobs lists
result = []
start = 0

for i in range(num_jobs):
    # Add an extra element to the first few lists to account for remainder
    end = start + chunk_size + (1 if i < remainder else 0)
    result.append(train_data_id_list[start:end])
    start = end

# Check the result
"""for i, sublist in enumerate(result):
    print(f"List {i + 1} (length {len(sublist)}): {sublist}")"""

'for i, sublist in enumerate(result):\n    print(f"List {i + 1} (length {len(sublist)}): {sublist}")'

In [8]:
def get_hours(time_of_day):
    ts = int(time_of_day)
    ts /= 1000
    date_var = datetime.utcfromtimestamp(ts).strftime('%H:%M:%S')
    return int(date_var.split(':')[0])

def get_mins(time_of_day):
    ts = int(time_of_day)
    ts /= 1000
    date_var = datetime.utcfromtimestamp(ts).strftime('%H:%M:%S')
    return int(date_var.split(':')[1])
    
def get_secs(time_of_day):
    ts = int(time_of_day)
    ts /= 1000
    date_var = datetime.utcfromtimestamp(ts).strftime('%H:%M:%S')
    return int(date_var.split(':')[2])

In [9]:
def transform_parquet_file(parquet_file_path, file_id):

    directory = Path(parquet_file_path)

    # Check if any Parquet file exists in the directory
    parquet_files = list(directory.glob("*.parquet"))

    if parquet_files:
        parquet_df = pl.read_parquet(parquet_file_path+"*.parquet")
        parquet_df = parquet_df.drop(['id'])
        parquet_df = parquet_df.with_columns([
            pl.lit(file_id).alias("id"),
            pl.col("time_of_day").map_elements(get_hours, return_dtype=pl.Int32).alias("hour_of_day")
        ])

        parquet_df = parquet_df.drop(['time_of_day'])
        parquet_df = parquet_df.group_by(["id","weekday","hour_of_day","relative_date_PCIAT"])\
        .agg([pl.mean("enmo").name.suffix("_avg"), pl.mean("anglez").name.suffix("_avg"), pl.mean("light").name.suffix("_avg")\
              , pl.mean("battery_voltage").name.suffix("_avg")])
    else:
        columns = [("id", pl.String), ("weekday", pl.Int8), ("hour_of_day", pl.Int32), ("relative_date_PCIAT", pl.Float32)\
                  , ("enmo_avg", pl.Float32), ("anglez_avg", pl.Float32), ("light_avg", pl.Float32)\
                   , ("battery_voltage_avg", pl.Float32)]

        parquet_df = pl.DataFrame(schema=columns)
    
    return parquet_df

In [10]:
def unify_files(list_of_ids):
    df_initializing_id = list_of_ids.pop()
    parquet_file_name = "id={0}/".format(df_initializing_id)
    first_parquet_file = config_dict['train_parquet_root_path']+parquet_file_name
    first_parquet_df = transform_parquet_file(first_parquet_file, df_initializing_id)
    
    train_data_of_id = train_data.filter(pl.col('id')==df_initializing_id)
    first_joined_features_df = train_data_of_id.join(first_parquet_df, on="id", how="left")
    
    for id in tqdm(list_of_ids):
        parquet_file_name = "id={0}/".format(id)
        parquet_file_path = config_dict['train_parquet_root_path']+parquet_file_name
        parquet_df = transform_parquet_file(parquet_file_path, id)
        
        train_data_of_id = train_data.filter(pl.col('id')==id)
        joined_features = train_data_of_id.join(parquet_df, on="id", how="left")
        
        try:
            first_joined_features_df = pl.concat([first_joined_features_df, joined_features])
        except:
            print(id)
    return first_joined_features_df

In [13]:
parts_of_parquet = (Parallel(n_jobs=num_jobs)(delayed(unify_files)(id_list) for id_list in result))

100%|██████████| 42/42 [00:58<00:00,  1.40s/it]


In [13]:
len(parts_of_parquet)

92

In [14]:
for i in range(len(parts_of_parquet)):
    print(parts_of_parquet[i].shape)

(3204, 89)
(7472, 89)
(7629, 89)
(7901, 89)
(6563, 89)
(9645, 89)
(7501, 89)
(4982, 89)
(7391, 89)
(3209, 89)
(4777, 89)
(5766, 89)
(8381, 89)
(7686, 89)
(7451, 89)
(6142, 89)
(8901, 89)
(7535, 89)
(5613, 89)
(8417, 89)
(6893, 89)
(6530, 89)
(7654, 89)
(8267, 89)
(6475, 89)
(9323, 89)
(3563, 89)
(8536, 89)
(8287, 89)
(6617, 89)
(6495, 89)
(4962, 89)
(7865, 89)
(2749, 89)
(5208, 89)
(7309, 89)
(7299, 89)
(4920, 89)
(5554, 89)
(6693, 89)
(7078, 89)
(6957, 89)
(3612, 89)
(5554, 89)
(3134, 89)
(4511, 89)
(6146, 89)
(5713, 89)
(6098, 89)
(7340, 89)
(7215, 89)
(5289, 89)
(7420, 89)
(4643, 89)
(9225, 89)
(3226, 89)
(5378, 89)
(9822, 89)
(7641, 89)
(8201, 89)
(8717, 89)
(7198, 89)
(6719, 89)
(7961, 89)
(8219, 89)
(4340, 89)
(4358, 89)
(4449, 89)
(6806, 89)
(5457, 89)
(7838, 89)
(6925, 89)
(8697, 89)
(7163, 89)
(7550, 89)
(2874, 89)
(4497, 89)
(5619, 89)
(6318, 89)
(3034, 89)
(6671, 89)
(6251, 89)
(8525, 89)
(4927, 89)
(6817, 89)
(5993, 89)
(6336, 89)
(6927, 89)
(9548, 89)
(7720, 89)
(7501, 89)

In [15]:
test_df = parts_of_parquet[0]
#dd.sql("select * from test_df limit 100").pl()

In [16]:
from sklearn.preprocessing import StandardScaler

test_df_pandas = test_df.to_pandas()
complete_column = "id"

missing_mask = test_df_pandas.drop(columns=[complete_column]).isna()

for col in missing_mask.columns:
    test_df_pandas[col] = test_df_pandas[col].fillna(test_df_pandas[col].mean())

test_df_pandas.head(100)

,id,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,...,encoded_PAQ_A-Season,encoded_PAQ_C-Season,encoded_SDS-Season,weekday,hour_of_day,relative_date_PCIAT,enmo_avg,anglez_avg,light_avg,battery_voltage_avg
0,023eba7b,16,0,61.940529,21.937701,66.5,138.0,31.000000,70.000000,99.000000,...,3,4,4,3.980101,11.499368,56.162666,0.026015,-8.368335,28.082163,3868.136475
1,00008ff9,5,0,51.000000,16.877316,46.0,50.8,26.846154,63.538413,82.602194,...,4,4,4,3.980101,11.499368,56.162666,0.026015,-8.368335,28.082163,3868.136475
2,000fd460,9,0,61.940529,14.035590,48.0,46.0,22.000000,75.000000,70.000000,...,4,0,0,3.980101,11.499368,56.162666,0.026015,-8.368335,28.082163,3868.136475
3,00105258,10,1,71.000000,16.648696,56.5,75.6,26.846154,65.000000,94.000000,...,4,2,0,3.980101,11.499368,56.162666,0.026015,-8.368335,28.082163,3868.136475
4,00115b9f,9,0,71.000000,18.292347,56.0,81.6,26.846154,60.000000,97.000000,...,4,3,2,6.000000,16.000000,71.000000,0.000354,84.839874,2.919743,3914.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,00115b9f,9,0,71.000000,18.292347,56.0,81.6,26.846154,60.000000,97.000000,...,4,3,2,7.000000,9.000000,79.000000,0.020218,73.676086,1.346589,3854.946289
96,00115b9f,9,0,71.000000,18.292347,56.0,81.6,26.846154,60.000000,97.000000,...,4,3,2,7.000000,17.000000,79.000000,0.025953,72.662369,1.347945,3854.926514
97,00115b9f,9,0,71.000000,18.292347,56.0,81.6,26.846154,60.000000,97.000000,...,4,3,2,1.000000,15.000000,80.000000,0.019209,21.315845,5.312884,3848.493896
98,00115b9f,9,0,71.000000,18.292347,56.0,81.6,26.846154,60.000000,97.000000,...,4,3,2,3.000000,18.000000,47.000000,0.058393,-16.245419,34.401844,4093.626221


In [19]:
import pandas as pd
scaler = StandardScaler()

df_scaled_values = scaler.fit_transform(test_df_pandas.drop(columns=[complete_column]))
df_scaled = pd.DataFrame(df_scaled_values, columns=test_df_pandas.drop(columns=[complete_column]).columns)

# Add the complete column back to the scaled DataFrame
df_scaled[complete_column] = test_df_pandas[complete_column].values
df_scaled = df_scaled[test_df_pandas.columns]  # Ensure original column order

df_scaled.head(100)

,id,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,...,encoded_PAQ_A-Season,encoded_PAQ_C-Season,encoded_SDS-Season,weekday,hour_of_day,relative_date_PCIAT,enmo_avg,anglez_avg,light_avg,battery_voltage_avg
0,023eba7b,1.884152,-0.473112,9.546355e-16,-0.033265,1.724340,0.748998,1.909039e+01,1.268161,1.684483e+00,...,-10.303424,1.268610,1.769989,0.000000,0.000000,1.937035e-08,3.903569e-08,-2.292249e-09,-1.390235e-09,8.134315e-08
1,00008ff9,-2.028241,-0.473112,-1.469893e+00,-0.893555,-3.604611,-1.904877,-1.632769e-14,0.000000,1.459826e-15,...,0.038735,1.268610,1.769989,0.000000,0.000000,1.937035e-08,3.903569e-08,-2.292249e-09,-1.390235e-09,8.134315e-08
2,000fd460,-0.605553,-0.473112,9.546355e-16,-1.376662,-3.084713,-2.050962,-2.227213e+01,2.249470,-1.294575e+00,...,0.038735,-1.610530,-1.460455,0.000000,0.000000,1.937035e-08,3.903569e-08,-2.292249e-09,-1.390235e-09,8.134315e-08
3,00105258,-0.249881,2.113664,1.217167e+00,-0.932422,-0.875148,-1.150105,-1.632769e-14,0.286853,1.170852e+00,...,0.038735,-0.170960,-1.460455,0.000000,0.000000,1.937035e-08,3.903569e-08,-2.292249e-09,-1.390235e-09,8.134315e-08
4,00115b9f,-0.605553,-0.473112,1.217167e+00,-0.652993,-1.005123,-0.967499,-1.632769e-14,-0.694455,1.479031e+00,...,0.038735,0.548825,0.154767,1.032355,0.655841,5.859069e-01,-1.008391e+00,2.115308e+00,-4.358102e-01,2.497957e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,00115b9f,-0.605553,-0.473112,1.217167e+00,-0.652993,-1.005123,-0.967499,-1.632769e-14,-0.694455,1.479031e+00,...,0.038735,0.548825,0.154767,1.543447,-0.364213,9.018165e-01,-2.277988e-01,1.861952e+00,-4.630571e-01,-7.184024e-02
96,00115b9f,-0.605553,-0.473112,1.217167e+00,-0.652993,-1.005123,-0.967499,-1.632769e-14,-0.694455,1.479031e+00,...,0.038735,0.548825,0.154767,1.543447,0.801564,9.018165e-01,-2.426615e-03,1.838946e+00,-4.630336e-01,-7.194795e-02
97,00115b9f,-0.605553,-0.473112,1.217167e+00,-0.652993,-1.005123,-0.967499,-1.632769e-14,-0.694455,1.479031e+00,...,0.038735,0.548825,0.154767,-1.523106,0.510119,9.413052e-01,-2.674678e-01,6.736657e-01,-3.943613e-01,-1.069832e-01
98,00115b9f,-0.605553,-0.473112,1.217167e+00,-0.652993,-1.005123,-0.967499,-1.632769e-14,-0.694455,1.479031e+00,...,0.038735,0.548825,0.154767,-0.500922,0.947286,-3.618217e-01,1.272379e+00,-1.787660e-01,1.094561e-01,1.228130e+00


In [73]:
# Parameters for the MICE-like imputation
iterations = 17  # Number of iterations for imputation
epochs = 35  # Training epochs for each imputer model
BATCH_SIZE = 16 * tpu_strategy.num_replicas_in_sync  # Batch size for training


# Function to create a simple neural network model for imputation
def create_imputer_model(input_dim):
    df_input = tf.keras.layers.Input(shape=(input_dim,), name="df_input")
    hidden_layer1 = tf.keras.layers.Dense(85, activation="relu")(df_input)
    hidden_layer2 = tf.keras.layers.Dense(65, activation="relu")(hidden_layer1)
    hidden_layer3 = tf.keras.layers.Dense(35, activation="relu")(hidden_layer2)
    df_output = tf.keras.layers.Dense(1)(hidden_layer3)
    
    """model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_dim,), name="df_input"),
        tf.keras.layers.Dense(85, activation="relu"),
        tf.keras.layers.Dense(65, activation="relu"),
        tf.keras.layers.Dense(35, activation="relu"),
        tf.keras.layers.Dense(1)
    ])"""
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    
    model = tf.keras.Model(df_input, df_output)
    model.compile(optimizer="adamax", loss="mean_squared_error")
    return model

In [69]:
import numpy as np

col = 'Physical-BMI'
missing_rows = missing_mask[col]

if not missing_rows.any():
    print("no missing data; skipping over")
else:
    X_train = df_scaled.loc[~missing_rows].drop(columns=[col, complete_column]).values
    y_train = df_scaled.loc[~missing_rows, col].values

    train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    train_ds = train_ds.batch(batch_size=BATCH_SIZE, drop_remainder=True)
    train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
    
    X_pred = df_scaled.loc[missing_rows].drop(columns=[col, complete_column]).values
    pred_ds = np.array(X_pred)

In [76]:
train_ds.take(1)

<_TakeDataset element_spec=(TensorSpec(shape=(128, 87), dtype=tf.float64, name=None), TensorSpec(shape=(128,), dtype=tf.float64, name=None))>

In [77]:
pred_ds = tf.data.Dataset.from_tensor_slices(X_pred)
pred_ds = pred_ds.batch(batch_size=BATCH_SIZE, drop_remainder=False)
pred_ds = pred_ds.prefetch(tf.data.AUTOTUNE)

In [78]:
pred_ds.take(1)

<_TakeDataset element_spec=TensorSpec(shape=(None, 87), dtype=tf.float64, name=None)>

In [74]:
with tpu_strategy.scope():
    imputer_model = create_imputer_model(input_dim=X_train.shape[1])
    imputer_model.fit(train_ds, epochs=epochs, callbacks=[early_stopping_cb])

Epoch 1/35


I0000 00:00:1730540787.482500     887 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(90903836f2f76940:0:0), session_name()


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.5596
Epoch 2/35


I0000 00:00:1730540787.795673     887 tpu_compile_op_common.cc:245] Compilation of 90903836f2f76940:0:0 with session name  took 313.114924ms and succeeded
I0000 00:00:1730540787.797981     887 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(90903836f2f76940:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_9280048792843290998", property.function_library_fingerprint = 7072008580332376410, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1730540787.798007     887 tpu_compilation_cache_interface.cc:541] After adding entry for key 90903836f2f76940:0:0

20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.1727 
Epoch 3/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0723 
Epoch 4/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0382 
Epoch 5/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0324 
Epoch 6/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0265
Epoch 7/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0218 
Epoch 8/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0180 
Epoch 9/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0144 
Epoch 10/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0118  
Epoch 11/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0099 
Epoch 12/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0085 
Epoch 13/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0073 
Epoch 14/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0066
Epoch 15/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0057 
Epoch 16/35
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0057 
Epoch 17/35
20/

In [79]:
with tpu_strategy.scope():
    predicted_values = imputer_model.predict(pred_ds)

df_scaled.loc[missing_rows, col] = predicted_values

3/5 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step 

2024-11-02 09:51:28.005008: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node functional_7_1/dense_35_1/Add/ReadVariableOp.
I0000 00:00:1730541088.033562     920 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(6a42994698299027:0:0), session_name()
I0000 00:00:1730541088.091666     920 tpu_compile_op_common.cc:245] Compilation of 6a42994698299027:0:0 with session name  took 58.033494ms and succeeded
I0000 00:00:1730541088.092102     920 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(6a42994698299027:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_data_distributed_10397657841767457932", property.function_library_fingerprint = 9462983159416133888, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 111ms/step


I0000 00:00:1730541088.463789     878 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(adafece44b98396f:0:0), session_name()
I0000 00:00:1730541088.533937     878 tpu_compile_op_common.cc:245] Compilation of adafece44b98396f:0:0 with session name  took 70.110512ms and succeeded
I0000 00:00:1730541088.534427     878 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(adafece44b98396f:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_data_distributed_17672165994022151004", property.function_library_fingerprint = 14192388935307804051, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "4,87,;", property.guaranteed_constants_size = 0, embedding

In [80]:
df_scaled

,id,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,...,encoded_PAQ_A-Season,encoded_PAQ_C-Season,encoded_SDS-Season,weekday,hour_of_day,relative_date_PCIAT,enmo_avg,anglez_avg,light_avg,battery_voltage_avg
0,023eba7b,1.884152,-0.473112,9.546355e-16,-0.033265,1.724340e+00,7.489980e-01,1.909039e+01,1.268161,1.684483e+00,...,-10.303424,1.268610,1.769989,0.000000,0.000000,1.937035e-08,3.903569e-08,-2.292249e-09,-1.390235e-09,8.134315e-08
1,00008ff9,-2.028241,-0.473112,-1.469893e+00,-0.893555,-3.604611e+00,-1.904877e+00,-1.632769e-14,0.000000,1.459826e-15,...,0.038735,1.268610,1.769989,0.000000,0.000000,1.937035e-08,3.903569e-08,-2.292249e-09,-1.390235e-09,8.134315e-08
2,000fd460,-0.605553,-0.473112,9.546355e-16,-1.376662,-3.084713e+00,-2.050962e+00,-2.227213e+01,2.249470,-1.294575e+00,...,0.038735,-1.610530,-1.460455,0.000000,0.000000,1.937035e-08,3.903569e-08,-2.292249e-09,-1.390235e-09,8.134315e-08
3,00105258,-0.249881,2.113664,1.217167e+00,-0.932422,-8.751481e-01,-1.150105e+00,-1.632769e-14,0.286853,1.170852e+00,...,0.038735,-0.170960,-1.460455,0.000000,0.000000,1.937035e-08,3.903569e-08,-2.292249e-09,-1.390235e-09,8.134315e-08
4,00115b9f,-0.605553,-0.473112,1.217167e+00,-0.652993,-1.005123e+00,-9.674990e-01,-1.632769e-14,-0.694455,1.479031e+00,...,0.038735,0.548825,0.154767,1.032355,0.655841,5.859069e-01,-1.008391e+00,2.115308e+00,-4.358102e-01,2.497957e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3199,0219f64a,-1.672569,-0.473112,8.141082e-01,-0.978496,-2.954739e+00,-1.783139e+00,-1.308045e+01,0.679377,-1.645872e-01,...,0.038735,1.268610,-0.652844,0.000000,0.000000,1.937035e-08,3.903569e-08,-2.292249e-09,-1.390235e-09,8.134315e-08
3200,021b2020,-0.605553,2.113664,9.546355e-16,-0.255356,-1.654995e+00,-8.944566e-01,5.302887e+00,0.090592,-1.645872e-01,...,0.038735,0.548825,1.769989,0.000000,0.000000,1.937035e-08,3.903569e-08,-2.292249e-09,-1.390235e-09,8.134315e-08
3201,0232fd98,1.172808,2.113664,9.546355e-16,-0.092353,-1.847048e-15,4.324980e-16,-1.632769e-14,0.000000,1.459826e-15,...,0.038735,1.268610,-1.460455,0.000000,0.000000,1.937035e-08,3.903569e-08,-2.292249e-09,-1.390235e-09,8.134315e-08
3202,02347f13,-0.249881,-0.473112,9.546355e-16,-0.356940,-2.252760e-01,-4.318545e-01,5.302887e+00,3.034516,5.544957e-01,...,0.038735,-1.610530,-1.460455,0.000000,0.000000,1.937035e-08,3.903569e-08,-2.292249e-09,-1.390235e-09,8.134315e-08


In [82]:
df_scaled[test_df_pandas.drop(columns=[complete_column]).columns] \
= scaler.inverse_transform(df_scaled[test_df_pandas.drop(columns=[complete_column]).columns])

df_imputed = df_scaled

In [83]:
df_imputed

,id,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,...,encoded_PAQ_A-Season,encoded_PAQ_C-Season,encoded_SDS-Season,weekday,hour_of_day,relative_date_PCIAT,enmo_avg,anglez_avg,light_avg,battery_voltage_avg
0,023eba7b,16.0,0.0,61.940529,21.937701,66.500000,138.000000,31.000000,70.000000,99.000000,...,3.0,4.0,4.0,3.980101,11.499368,56.162666,0.026015,-8.368335,28.082163,3868.136475
1,00008ff9,5.0,0.0,51.000000,16.877316,46.000000,50.800000,26.846154,63.538413,82.602194,...,4.0,4.0,4.0,3.980101,11.499368,56.162666,0.026015,-8.368335,28.082163,3868.136475
2,000fd460,9.0,0.0,61.940529,14.035590,48.000000,46.000000,22.000000,75.000000,70.000000,...,4.0,0.0,0.0,3.980101,11.499368,56.162666,0.026015,-8.368335,28.082163,3868.136475
3,00105258,10.0,1.0,71.000000,16.648696,56.500000,75.600000,26.846154,65.000000,94.000000,...,4.0,2.0,0.0,3.980101,11.499368,56.162666,0.026015,-8.368335,28.082163,3868.136475
4,00115b9f,9.0,0.0,71.000000,18.292347,56.000000,81.600000,26.846154,60.000000,97.000000,...,4.0,3.0,2.0,6.000000,16.000000,71.000000,0.000354,84.839874,2.919743,3914.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3199,0219f64a,6.0,0.0,68.000000,16.377681,48.500000,54.800000,24.000000,67.000000,81.000000,...,4.0,4.0,1.0,3.980101,11.499368,56.162666,0.026015,-8.368335,28.082163,3868.136475
3200,021b2020,9.0,1.0,61.940529,20.631322,53.500000,84.000000,28.000000,64.000000,81.000000,...,4.0,3.0,4.0,3.980101,11.499368,56.162666,0.026015,-8.368335,28.082163,3868.136475
3201,0232fd98,14.0,1.0,61.940529,21.590137,59.866617,113.389711,26.846154,63.538413,82.602194,...,4.0,4.0,0.0,3.980101,11.499368,56.162666,0.026015,-8.368335,28.082163,3868.136475
3202,02347f13,10.0,0.0,61.940529,20.033783,59.000000,99.200000,28.000000,79.000000,88.000000,...,4.0,0.0,0.0,3.980101,11.499368,56.162666,0.026015,-8.368335,28.082163,3868.136475


In [85]:
df_imputed["Physical-BMI"].isna().any()

False

In [50]:
with tpu_strategy.scope():
    for iteration in range(iterations):
        print(f"\nIteration {iteration + 1}/{iterations}")

        # Iterate over each column to impute, excluding the complete column
        for col in df_scaled.columns:
            print("Starting with column --> {0}".format(col))

            if col == complete_column:
                print("complete column; skipping over")
                continue  # Skip the complete column

            # Use the original missing mask to find rows with missing values
            missing_rows = missing_mask[col]

            # If no missing values, skip this column
            if not missing_rows.any():
                print("no missing data; skipping over")
                continue

            print("training for column --> {0}".format(col))
            # Define training data using columns other than the target column
            X_train = df_scaled.loc[~missing_rows].drop(columns=[col, complete_column]).values
            y_train = df_scaled.loc[~missing_rows, col].values

            # Define data to be predicted (rows where target column has missing values)
            X_pred = df_scaled.loc[missing_rows].drop(columns=[col, complete_column]).values

            X_train = np.array(X_train, dtype=np.float32)
            y_train = np.array(y_train, dtype=np.float32)
            X_pred = np.array(X_pred, dtype=np.float32)

            # Create and train the imputer model
            imputer_model = create_imputer_model(input_dim=X_train.shape[1])
            imputer_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

            # Predict missing values and update the DataFrame
            predicted_values = imputer_model.predict(X_pred).flatten()
            df_scaled.loc[missing_rows, col] = predicted_values


Iteration 1/17
Starting with column --> id
complete column; skipping over
Starting with column --> Basic_Demos-Age
no missing data; skipping over
Starting with column --> Basic_Demos-Sex
no missing data; skipping over
Starting with column --> CGAS-CGAS_Score
training for column --> CGAS-CGAS_Score


/usr/local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-11-01 15:04:19.711129: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node StatefulPartitionedCall.
I0000 00:00:1730473459.956182     859 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(d24346abc4149311:0:0), session_name()
I0000 00:00:1730473460.364629     859 tpu_compile_op_common.cc:245] Compilation of d24346abc4149311:0:0 with session name  took 408.406527ms and succeeded
I0000 00:00:1730473460.367795     859 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(d24346abc4149311:0:0), s

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 856ms/step


I0000 00:00:1730473515.666396     908 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(85804c023a0c4a7b:0:0), session_name()
I0000 00:00:1730473515.756172     908 tpu_compile_op_common.cc:245] Compilation of 85804c023a0c4a7b:0:0 with session name  took 89.731831ms and succeeded
I0000 00:00:1730473515.757146     908 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(85804c023a0c4a7b:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_data_distributed_17185690794720253171", property.function_library_fingerprint = 3249260976836082871, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "4,87,;", property.guaranteed_constants_size = 0, embedding_

Starting with column --> Physical-BMI
training for column --> Physical-BMI


I0000 00:00:1730473519.257417     854 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(a06a2b99333b8f68:0:0), session_name()
I0000 00:00:1730473519.646939     854 tpu_compile_op_common.cc:245] Compilation of a06a2b99333b8f68:0:0 with session name  took 389.482975ms and succeeded
I0000 00:00:1730473519.650210     854 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(a06a2b99333b8f68:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_7158594234667614164", property.function_library_fingerprint = 14287038548144303800, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "63,87,;63,;", property.guaranteed_constants_size = 0, embedding_pa

 3/17 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step  

2024-11-01 15:06:13.841244: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node sequential_6_1/dense_30_1/Add/ReadVariableOp.
I0000 00:00:1730473573.870072     872 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(dc71e40bb76de3e8:0:0), session_name()
I0000 00:00:1730473573.934670     872 tpu_compile_op_common.cc:245] Compilation of dc71e40bb76de3e8:0:0 with session name  took 64.561319ms and succeeded
I0000 00:00:1730473573.935205     872 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(dc71e40bb76de3e8:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_data_distributed_1475694961096069229", property.function_library_fingerprint = 6330706768104190395, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2,

15/17 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step

TypeError: '>' not supported between instances of 'int' and 'NoneType'